## **Data Anonymization**

In [ ]:
%pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from tqdm import tqdm
import spacy

#### Reading dataset

In [ ]:
df = pd.read_json("hf://datasets/AGBonnet/augmented-clinical-notes/augmented_notes_30K.jsonl", lines=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
df.head()

,note,conversation,idx,summary,full_note
0,"A a sixteen year-old girl, presented to our Ou...","Doctor: Good morning, what brings you to the O...",155216,"{\n""visit motivation"": ""Discomfort in the neck...","A a sixteen year-old girl, presented to our Ou..."
1,This is the case of a 56-year-old man that was...,"Doctor: Hi, how are you feeling today?\nPatien...",77465,"{\n""visit motivation"": ""Complaints of a dull p...",This is the case of a 56-year-old man that was...
2,A 36-year old female patient visited our hospi...,"Doctor: Hello, what brings you to the hospital...",133948,"{\n""visit motivation"": ""Pain and restricted ra...",A 36-year old female patient visited our hospi...
3,A 49-year-old male presented with a complaint ...,"Doctor: Good morning, Mr. [Patient's Name]. I'...",80176,"{\n""visit motivation"": ""Pain in the left proxi...",A 49-year-old male presented with a complaint ...
4,A 47-year-old male patient was referred to the...,"Doctor: Good morning, how are you feeling toda...",72232,"{\n""visit motivation"": ""Recurrent attacks of p...",A 47-year-old male patient was referred to the...


In [ ]:
df.dropna(inplace=True)

In [ ]:
# Remove unwanted characters from the 'note' and 'full note' columns
df['note'] = df['note'].astype(str).str.replace(r'(\\n|\n|\r|\[|\]|\(|\))', '', regex=True)
df['full_note'] = df['full_note'].astype(str).str.replace(r'(\\n|\n|\r|\[|\]|\(|\))', '', regex=True)

In [ ]:
len(df)

30000

In [ ]:
# gender to mask
gender_terms = {
    "male", "female", "man", "woman", "boy", "girl",
    "he", "she", "his", "her", "him", "hers", "himself", "herself"
}

def clean_duplicate_labels(text):
    # Remove consecutive [PROBLEM] or [TREATMENT]
    text = re.sub(r'(?:\s*\[PROBLEM\]\s*){2,}', ' [PROBLEM] ', text)
    text = re.sub(r'(?:\s*\[TREATMENT\]\s*){2,}', ' [TREATMENT] ', text)

    # Remove multiple spaced
    text = re.sub(r'\s{2,}', ' ', text)
    return text.strip()

### **CLINICAL-BERT NER**

In [ ]:
model_name = "samrawal/bert-base-uncased_clinical-ner"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

# Output structure
df_anonym_text = {'index': [], 'note': [], 'anonymized_note': [], 'full_note': [], 'anonymized_full_note': []}

# Iterate over all dataset
for index, row in tqdm(df.iterrows(), total=len(df), desc="Anonymized dataset"):
    note = row["note"]
    full_note = row["full_note"]

    df_anonym_text['index'].append(index)
    df_anonym_text['note'].append(note)
    df_anonym_text['full_note'].append(full_note)

    for idx, text in enumerate([note, full_note]):
        ner_results = ner_pipeline(text)

        # Build masked text by replacing spans
        anonym_text = ""
        i = 0
        for entity in sorted(ner_results, key=lambda x: x['start']):
            start, end = entity['start'], entity['end']
            label = entity['entity_group']

            if label in {"problem", "treatment"}:
                # Append non-entity text
                anonym_text += text[i:start]
                # Append label placeholder
                anonym_text += f"[{label.upper()}]"
                i = end

        anonym_text += text[i:]

        anonym_text = clean_duplicate_labels(anonym_text)

        if idx == 0:
            df_anonym_text['anonymized_note'].append(anonym_text)
        else:
            df_anonym_text['anonymized_full_note'].append(anonym_text)

tokenizer_config.json:   0%|          | 0.00/300 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/895 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Device set to use cuda:0
Anonymized dataset:   0%|          | 0/30000 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Anonymized dataset: 100%|██████████| 30000/30000 [47:53<00:00, 10.44it/s]


In [ ]:
df_anonym = pd.DataFrame(df_anonym_text)

In [ ]:
df_anonym['anonymized_note'][0]

'A a sixteen year-old girl, presented to our Outpatient department with the complaints of [PROBLEM] as well as [PROBLEM]. She was not able to maintain an erect posture and would tend to fall on either side while standing up from a sitting position. She would keep her head turned to the right and upwards due to [PROBLEM]. There was [PROBLEM] in the lumbar region. To counter [PROBLEM], she would keep her limbs in a specific position to allow her body weight to be supported. Due to [PROBLEM] restrictions with the body movements at the neck and in the lumbar region, she would require assistance in standing and walking. She would require her parents to help her with daily chores, including all activities of self-care.\nShe had been experiencing [PROBLEM] for the past four months since when she was introduced to [TREATMENT] for the control of [PROBLEM]. This was not her first experience with [TREATMENT] over the past seven years since she had been diagnosed with [PROBLEM]. Her first episode 

### **SPACY NER**

In [ ]:
spacy.cli.download("en_core_web_lg")

# spaCy model
nlp = spacy.load("en_core_web_lg")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import re
from tqdm import tqdm

# Dictionary mapping short entity labels to full descriptive names in English
entity_labels_full = {
    "PERSON": "PERSON",
    "NORP": "NATIONALITY/RELIGIOUS/POLITICAL GROUP",
    "DATE": "DATE",
    "CARDINAL": "CARDINAL NUMBER",
    "QUANTITY": "QUANTITY",
    "FAC": "FACILITY",
    "ORG": "ORGANIZATION",
    "GPE": "GEO-POLITICAL ENTITY",
    "LOC": "LOCATION",
    "LANGUAGE": "LANGUAGE",
    "TIME": "TIME"
}

entities_to_mask = set(entity_labels_full.keys())

for index, row in tqdm(df_anonym.iterrows(), total=len(df_anonym), desc="Anonymized dataset"):
    note = row["anonymized_note"]
    full_note = row["anonymized_full_note"]

    for idx, text in enumerate([note, full_note]):

        doc = nlp(text)

        # Collect entities to replace with their full label in square brackets
        to_replace = [(ent.start_char, ent.end_char, f"[{entity_labels_full[ent.label_]}]")
                      for ent in doc.ents if ent.label_ in entities_to_mask]

        # Replace entities from the end to avoid messing up indices
        replaced_text = text
        for start, end, full_label in sorted(to_replace, key=lambda x: x[0], reverse=True):
            replaced_text = replaced_text[:start] + full_label + replaced_text[end:]

        # Gender
        replaced_text = re.sub(
            r'\b\w+\b',
            lambda m: "[GENDER]" if m.group().lower() in gender_terms else m.group(),
            replaced_text
        )

        if idx == 0:
            df_anonym.at[index, 'anonymized_note'] = replaced_text
        else:
            df_anonym.at[index, 'anonymized_full_note'] = replaced_text

Anonymized dataset: 100%|██████████| 1/1 [00:00<00:00,  3.38it/s]


In [ ]:
df_anonym['anonymized_note'][0]

'A a [DATE] [GENDER], presented to our [DATE] department with the complaints of [PROBLEM] as well as [PROBLEM]. [GENDER] was not able to maintain an erect posture and would tend to fall on either side while standing up from a sitting position. [GENDER] would keep [GENDER] head turned to the right and upwards due to [PROBLEM]. There was [PROBLEM] in the lumbar region. To counter [PROBLEM], [GENDER] would keep [GENDER] limbs in a specific position to allow [GENDER] body weight to be supported. Due to [PROBLEM] restrictions with the body movements at the neck and in the lumbar region, [GENDER] would require assistance in standing and walking. [GENDER] would require [GENDER] parents to help [GENDER] with daily chores, including all activities of self-care.\n[GENDER] had been experiencing [PROBLEM] for [DATE] since when [GENDER] was introduced to [TREATMENT] for the control of [PROBLEM]. This was not [GENDER] first experience with [TREATMENT] over [DATE] since [GENDER] had been diagnosed wi

In [ ]:
from google.colab import files

df_anonym.to_csv('anonymized_dataset.csv', index=False)
files.download('anonymized_dataset.csv')